In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
import jReversion as jR
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, balanced_accuracy_score
import networkx as nx
from statannot import add_stat_annotation

In [2]:
#load data
# network_name_list= [Grieco, TLGLNetwork, bauer, ERBB, fibroblast, saez, BLT, colitis_jijoo, prostate]
# integrated_data = pd.DataFrame()
Model = jR.grieco()
BooleanRuleFileName = '../' + Model['BooleanRule_filename']
NetworkName = Model['network_name']

NumInputs = Model['num_inputs']
NumInputConditions = Model['num_input_conditions']

InputConditions = Model['input_conditions']

OutputNodes = Model['output_nodes']
InputNodes = Model['input_nodes']

# Set parameters
# Note the node name for Gread is the index (integer), one can encode the nodename by adding prefix and suffix
# If the node name from the input file is not this simple, one need to create a dictionary to record the mapping
Prefix, Suffix = 'n', 'n'

TempI = jR.initialize(BooleanRuleFileName, Prefix, Suffix)
Mapping = TempI['mapping']
InverseMapping = TempI['inverse_mapping']
GRead = TempI['Gread']
ReadNodes = TempI['read_nodes']
if Model['reduction_required']:
    GRead = BDOIp.Get_reduced_network(GRead)

try:
    GExpanded = nx.read_gml('../networks/' + NetworkName + '_expanded_network.gml')
except IOError:
    GExpanded = BDOIp.Get_expanded_network(GRead, prefix=Prefix, suffix=Suffix)
    nx.write_gml(GExpanded, '../networks/' + NetworkName + '_expanded_network.gml')

TempGIOW = jR.get_input_output_relation(GExpanded, Mapping, InverseMapping, InputConditions, OutputNodes,
                                            constant_nodes=[])

IORelation = TempGIOW['io_relation']
    
table_original = pd.read_csv('../data/' + NetworkName + '_table_for_original_network.tsv', sep='\t').sort_values(by='node').set_index('node')
table_expanded_network = pd.read_csv('../data/' + NetworkName + '_table_for_expanded_network.tsv', sep='\t')

In [3]:
r1_list = table_expanded_network[table_expanded_network['Class_LDOI'] == 'R1']['node'].tolist()
print(r1_list)

['PLCG', '~PTEN', '~PDK1', 'MAP3K1_3', 'BCL2', '~PI3K', '~p70', '~CREB', 'SOS', 'AKT', '~MSK', 'PPP2CA', '~MAX', '~SPRY', '~FOXO3', '~DUSP1', 'FGFR3', 'RAS', 'AP1']


In [4]:
is_r2 = dict()
for r1 in r1_list:
    is_r2[r1] = jR.test_r2(GExpanded, Mapping, InverseMapping, OutputNodes, InputConditions, IORelation, r1)
    

In [5]:
print(is_r2)

{'PLCG': False, '~PTEN': False, '~PDK1': False, 'MAP3K1_3': False, 'BCL2': False, '~PI3K': True, '~p70': False, '~CREB': False, 'SOS': True, 'AKT': False, '~MSK': False, 'PPP2CA': False, '~MAX': False, '~SPRY': False, '~FOXO3': False, '~DUSP1': False, 'FGFR3': True, 'RAS': True, 'AP1': False}


In [6]:
jR.ineffective_test_ldoi(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions, IORelation, set(['~PDK1', 'p70']))

True

In [7]:
jR.ineffective_test_ldoi(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions, IORelation, set(['PLCG', '~RAS']))

True

In [8]:
jR.ineffective_test_ldoi(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions, IORelation, set(['~CREB', 'DUSP1']))

True

In [9]:
is_r2 = dict()
for r1 in r1_list:
    is_r2[r1] = jR.test_r2_ver2(GExpanded, Mapping, InverseMapping, OutputNodes, InputConditions, IORelation, r1, r1_list)

In [10]:
print(is_r2)

{'PLCG': True, '~PTEN': False, '~PDK1': True, 'MAP3K1_3': False, 'BCL2': False, '~PI3K': True, '~p70': False, '~CREB': True, 'SOS': True, 'AKT': False, '~MSK': False, 'PPP2CA': False, '~MAX': False, '~SPRY': False, '~FOXO3': False, '~DUSP1': False, 'FGFR3': True, 'RAS': True, 'AP1': False}


In [11]:
jR.ineffective_test_ldoi(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions, IORelation, set(['PPP2CA', 'p70']))

True

In [12]:
jR.ineffective_test_ldoi(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions, IORelation, set(['AP1', 'p70']))

True